In [1]:
import os
import shutil
import random
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers.legacy import Adam


# Enable Debug Mode for tf.data
tf.data.experimental.enable_debug_mode()

# Optionally force TensorFlow to run on the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

def split_dataset(source_dir, train_dir, val_dir, split_ratio=0.9):
    """Splits the dataset into training and validation sets."""
    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            random.shuffle(images)
            split_point = int(len(images) * split_ratio)

            train_images = images[:split_point]
            val_images = images[split_point:]

            # Create directories for train and validation sets
            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

            # Move the images to the respective directories
            for img in train_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))

            for img in val_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

    print("Dataset split completed!")


# Disable multi-threading in the data generator
def start_training():
    global class_indices

    source_dir = 'deeplearning images'
    train_dir = 'split_data/train'
    val_dir = 'split_data/validation'

    # Split the dataset if necessary
    split_dataset(source_dir, train_dir, val_dir)

    # Define the data augmentation and preprocessing (now only using rescale)
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
    validation_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

    # Store class indices globally
    class_indices = train_generator.class_indices
    print("Class Indices:", class_indices)

    # Load the ResNet50 model and fine-tune it
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(train_generator, validation_data=validation_generator, epochs=5, workers=1, use_multiprocessing=False)

    # Save the model
    model.save('best_model.h5')
    messagebox.showinfo("Info", "Training completed!")


def upload_and_classify_image():
    global class_indices  # Access the global class indices

    file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((224, 224))
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk

        # Preprocess the image
        img_array = keras_image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0

        # Load the model and make a prediction
        model = load_model('best_model.h5')
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]

        # Get the class label using the stored class indices
        if class_indices:
            class_labels = {v: k for k, v in class_indices.items()}
            class_name = class_labels.get(predicted_class, "Unknown")
        else:
            class_name = "Unknown"

        result_label.config(text=f"Predicted Class: {class_name}")
    else:
        messagebox.showwarning("Warning", "No file selected!")


# Tkinter application setup
app = tk.Tk()
app.title("Image Classification App")

# Image display area
image_label = tk.Label(app)
image_label.pack(pady=20)

# Button to start training
train_button = tk.Button(app, text="Start Training", command=start_training)
train_button.pack(pady=20)

# Button to upload and classify image
upload_button = tk.Button(app, text="Upload and Classify Image", command=upload_and_classify_image)
upload_button.pack(pady=20)

# Label to show the classification result
result_label = tk.Label(app, text="")
result_label.pack(pady=20)

# Run the app
app.mainloop()


Dataset split completed!
Found 46 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Class Indices: {'Galena': 0, 'Goethite': 1, 'Pyrite': 2}
Epoch 1/5


2024-08-22 16:16:58.669616: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 6s 4s/step - loss: 3.0128 - accuracy: 0.2609 - val_loss: 2.2424 - val_accuracy: 0.4000
Epoch 2/5


2024-08-22 16:17:04.352476: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_12 in the registry.
Traceback (most recent call last):

  File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 255, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_12 in the registry.


2024-08-22 16:17:04.352515: W tensorflow/core/kernels/data/generator_dataset_op.cc:108] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_12 in the registry.
Traceback (most recent call last):

  File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 255, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_12 in the registry.


	 [[{{n

2/2 [==============================] - 5s 5s/step - loss: 1.9091 - accuracy: 0.4130 - val_loss: 1.1497 - val_accuracy: 0.4000
Epoch 3/5
2/2 [==============================] - 6s 5s/step - loss: 1.2345 - accuracy: 0.3043 - val_loss: 1.3856 - val_accuracy: 0.2667
Epoch 4/5


...

2/2 [==============================] - 6s 5s/step - loss: 1.2532 - accuracy: 0.2609 - val_loss: 1.0618 - val_accuracy: 0.4000
Epoch 5/5
1/1 [==============================] - 1s 537ms/step
Dataset split completed!
Found 51 images belonging to 3 classes.
Found 20 images belonging to 3 classes.
Class Indices: {'Galena': 0, 'Goethite': 1, 'Pyrite': 2}
Epoch 1/5
2/2 [==============================] - 6s 4s/step - loss: 3.7045 - accuracy: 0.3922 - val_loss: 2.0283 - val_accuracy: 0.4000
Epoch 2/5


2024-08-22 16:19:58.302226: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_92 in the registry.
Traceback (most recent call last):

  File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 255, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_92 in the registry.


2024-08-22 16:19:58.302273: W tensorflow/core/kernels/data/generator_dataset_op.cc:108] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_92 in the registry.
Traceback (most recent call last):

  File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 255, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_92 in the registry.


	 [[{{n

2/2 [==============================] - 7s 6s/step - loss: 2.3583 - accuracy: 0.3137 - val_loss: 1.7532 - val_accuracy: 0.2500
Epoch 3/5
2/2 [==============================] - 7s 4s/step - loss: 1.8277 - accuracy: 0.2745 - val_loss: 1.4668 - val_accuracy: 0.2500
Epoch 4/5
2/2 [==============================] - 7s 4s/step - loss: 1.3591 - accuracy: 0.2745 - val_loss: 1.2133 - val_accuracy: 0.4000
Epoch 5/5
2/2 [==============================] - 7s 5s/step - loss: 1.2899 - accuracy: 0.4118 - val_loss: 1.2464 - val_accuracy: 0.4500


2024-08-22 16:20:27.184668: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_147 in the registry.
Traceback (most recent call last):

  File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 255, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_147 in the registry.


2024-08-22 16:20:27.184791: W tensorflow/core/kernels/data/generator_dataset_op.cc:108] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_147 in the registry.
Traceback (most recent call last):

  File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 255, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_147 in the registry.


	 [

1/1 [==============================] - 1s 518ms/step


NameError: name 'start_training' is not defined

a
